In [1]:
import pandas as pd
import numpy as np
import tqdm
from tqdm import tqdm

In [103]:
fpath = 'D:\AtomicHackData\\ru_syn_dataset.json'

In [104]:
dataset = pd.read_json(fpath)

Dataset include words without any synonyms

In [116]:
dataset1 = dataset[dataset['synonyms_count'] >= 1].copy(deep=True)

In [125]:
dataset1.reset_index(drop=True, inplace=True)

In [128]:
dataset1['word_clear'] = [''] * dataset1.shape[0]
dataset1['synonyms_clear'] = [''] * dataset1.shape[0]

In [138]:
markings = {'(прост.)', '(устар.)', '(офиц.)', '(книжн.)', '(высок.)', '(разг.)', '(бран.)', '(шутл.)', '(обл.)', '(спец.)'}

In [139]:
litter = {'чегокого л', 'чегокого н', 'когочего л', 'когочего н', 'чегокого-л', 'чегокого-н', 'когочего-л', 'когочего-н', 'чего л', 'чего н', 'кого л', 'кого н', 'чего-л', 'чего-н', 'кого-л', 'кого-н',\
    'чтокто л', 'чтокто н', 'кточто л', 'кточто н', 'чтокто-л', 'чтокто-н', 'кточто-л', 'кточто-н', 'что л', 'что н', 'кто л', 'кто н', 'что-л', 'что-н' 'кто-л', 'кто-н',\
    'чемукому л', 'чемукому н','комучему л','комучему н','чемукому-л','чемукому-н','комучему-л','комучему-н','чему л','чему н','кому л','кому н','чему-л','чему-н','кому-л','кому-н',\
    'чтокого л','чтокого н','когочто л', 'когочто н', 'чтокого-н', 'чтокого-л', 'когочто-л', 'когочто-н',\
    'чемкем л', 'чемкем н', 'кемчем л', 'кемчем н', 'чемкем-л', 'чемкем-н', 'кемчем-л', 'кемчем-н', 'чем л', 'чем н', 'кем л', 'кем н', 'чем-л', 'чем-н', 'кем-л', 'кем-н',\
    'чемком л', 'чемком н', 'комчем л', 'комчем н', 'чемком-л', 'чемком-н', 'комчем-л', 'комчем-н', 'ком л', 'ком н', 'ком-л', 'ком-н',\
    'чёмком л', 'чёмком н', 'комчём л', 'комчём н', 'чёмком-л', 'чёмком-н', 'комчём-л', 'комчём-н', 'чём л', 'чём н', 'чём-л', 'чём-н'}

# Cleaning word

In [140]:
import re

In [141]:
def word_cleaning(markings: set, litter: set, word: str) -> str:
    word = word.lower().strip()
    split_word = word.split()
    for lexem in split_word:
        if lexem in markings:
            split_word.remove(lexem)
    word1 = ' '.join(split_word)
    word_wo_punct = re.sub(r'[^\w\s-]', '', word1)
    split_word1 = word_wo_punct.split()
    for lexem in split_word1:
        if lexem in litter:
            split_word1.remove(lexem)
    res_word = '-'.join(split_word1)
    return res_word

In [146]:
def synonyms_cleaning(markings: set, litter: str, syn_list: list) -> list:
    clear_syn_list = []
    for syn in syn_list:
        clear_syn_list.append(word_cleaning(markings, litter, syn))
    return clear_syn_list

In [147]:
for index, row in tqdm(dataset1.iterrows()):
    dataset1.at[index, 'word_clear'] = word_cleaning(markings, litter, row['word'])
    dataset1.at[index, 'synonyms_clear'] = synonyms_cleaning(markings, litter, row['synonyms'])

337360it [00:33, 10074.33it/s]


In [156]:
df = dataset1.sort_values(by=['word_clear']).copy(deep=True)

In [4]:
df.reset_index(drop=True, inplace=True)

In [5]:
df

,word,synonyms,synonyms_count,word_clear,synonyms_clear
0,10-разрядный,[десятиразрядный],1,10-разрядный,[десятиразрядный]
1,12-разрядный,[двенадцатиразрядный],1,12-разрядный,[двенадцатиразрядный]
2,128-разрядный,[стодвадцативосьмиразрядный],1,128-разрядный,[стодвадцативосьмиразрядный]
3,14-разрядный,[четырнадцатиразрядный],1,14-разрядный,[четырнадцатиразрядный]
4,16-разрядный,[шестнадцатиразрядный],1,16-разрядный,[шестнадцатиразрядный]
...,...,...,...,...,...
337355,ётёх,"[развалины, брошенный дом]",2,ётёх,"[развалины, брошенный-дом]"
337356,ёухикко,[инструмент],1,ёухикко,[инструмент]
337357,ёфикатор,"[ёделя, ёдельник]",2,ёфикатор,"[ёделя, ёдельник]"
337358,ёхнувший,[сдвинувший],1,ёхнувший,[сдвинувший]


In [6]:
df.to_json('D:\AtomicHackData\\ru_syn_dataset_mod1.json', force_ascii=False)